모델을 개발하는 데 사용할 라이브러리는 어떤 애플리케이션에 적용하느냐에 따라 다르다.

In [1]:
import numpy as np
import pandas as pd

# 주피터 셸에서 모두 출력하는 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 12.1 판다스와 모델 코드의 인터페이스

모델을 개발하는 과정에서 중요한 단계는 특징을 선택하고 추출하는 피처 엔지니어링이다.  
(원시 데이터셋으로부터 모델링에 유용할 만한 정보를 추출하는 변환이나 분석 과정)

DataFrame을 넘파이 배열로 변환하려면 to_numpy 메서드를 이용한다.

In [2]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, -.25, -4.1, 0],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

data.columns
data.to_numpy()

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,-0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


Index(['x0', 'x1', 'y'], dtype='object')

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  , -0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

다시 DataFrame으로 되돌릴려면 ndarray와 선택적으로 열 이름 리스트를 넘겨서 생성한다.

In [3]:
df2 = pd.DataFrame(data.to_numpy(), columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,-0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


to_numpy 메서드는 데이터가 한 가지 타입(모두 숫자형 등)으로 이루어져 있다는 가정하에 사용 가능하다. (데이터가 여러가지 타입이면 ndarray 반환)

In [6]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

df3.to_numpy()

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,-0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, -0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

전체 열 중 일부만 사용하고 싶을 때는 loc를 이용해서 to_numpy를 사용하자.

DataFrame에서 넘파이 배열로 변환하고 모델 매개변수의 이름을 출력 테이블이나 Series의 열을 추가한다.  
(아니면 메타데이터 관리를 수동으로 직접 작업해야함)

In [7]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  , -0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [10]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,-0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


category 열을 더미 값으로 치환하고 싶으면 더미 값을 생성하고 category 열을 삭제한 다음 결과와 합쳐야한다.

In [12]:
dummies = pd.get_dummies(data.category, prefix='category', dtype=float)
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,-0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


# 12.2 patsy로 모델 생성하기

patsy는 통계 모델을 위한 파이써 라이브러리로 R이나 S 통계 프로그래밍 언너에서 사용하는 수식 문법과 유사한 형식의 문자열 기반 수식 문법을 제공한다.  
(statsmodels를 설치하면 patsy가 자동적으로 설치됨)

In [13]:
conda install statsmodels

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\mit06\anaconda3

  added / updated specs:
    - statsmodels


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2023.11.~ --> pkgs/main::ca-certificates-2023.08.22-haa95532_0 
  certifi            conda-forge/noarch::certifi-2023.11.1~ --> pkgs/main/win-64::certifi-2023.11.17-py311haa95532_0 





Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




patsy의 DesignMatrix 인스턴스는 몇 가지 추가 메타데이터가 포함된 넘파이 ndarray로 볼 수 있다.

Intercept는 최소제곱법 회귀와 같은 선형 모델을 위한 표현이다.

In [15]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, -.25, -4.1, 0],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

import patsy
y, X = patsy.dmatrices('y ~ x0 + x1', data)
y
X

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,-0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3  -0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [16]:
np.asarray(y)

np.asarray(X)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  , -0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

모델에 0을 터해서 절편을 제거한다.

In [19]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3  -0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [22]:
coef, resid ,_, _ = np.linalg.lstsq(X, y,rcond=None)
coef

coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

array([[ 0.34615797],
       [-0.14680695],
       [-0.43018724]])

Intercept    0.346158
x0          -0.146807
x1          -0.430187
dtype: float64

## 12.2.1 patsy 용법으로 데이터 변환하기

In [31]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

자주 사용하는 변수 변환으로는 표준화(평균 0, 분산 1)와 중심화(평균값을 뺌)가 있다. (pastsy에는 이럼 목적을 위한 내장 함수가 존재)

In [32]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.88
          1         -0.70711        0.86
          1          0.00000        0.62
          1          0.70711       -3.23
          1          1.41421        0.87
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

patsy.build_design_matrices 함수는 입력으로 사용되는 원본 데이터셋에서 저장한 정보를 사용해 출력 데이터에 변환을 적용할 수 있는 함수다.

In [33]:
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],
                         'x1': [3.1, -0.5, 0, 2.3],
                         'y': [1, 2, 3, 4]})
new_data

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.97
           1          2.82843        0.37
           1          3.53553        0.87
           1          4.24264        3.17
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

patsy 문법에 더하기 기호는 덧셈이 아니므로 데이터셋에서 이름으로 열을 추가하고 싶으면 I 함수라는 특수한 함수로 둘러싸야 한다.

In [34]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        2.75
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

## 12.2.2 범주형 데이터와 patsy

patsy에서 숫자가 아닌 용법을 사용하면 기본적으로 더미 변수로 변환된다.

Intercept가 존재한다면 공선성을 피하기 위해 레벨 중 하나는 남겨두게 된다.

In [37]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
                     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})
data

y, X = patsy.dmatrices('v2 ~ key1', data)
X

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

모델에서 Intercept를 생략하면 범줏값의 열은 모델 설계 배열에 포함된다.

In [38]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

숫자 열은 C 함수를 이용해서 범주형으로 해석할 수 있다.

In [39]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

모델에서 여러 범주형 항을 사용하는 경우 분산분석 모델에서처럼 key1:key2 같은 용법을 사용할 수 있게 되므로 더 복잡해진다.

In [40]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

patsy는 특정 순서에 따라 데이터를 변환하는 방법과 범주형 데이터를 변환하는 여러 방법을 제공한다. (자세한 내용은 온라인 문서 참조)

# 12.3 statsmodels 소개

statsmodels는 다양한 종류의 통계 모델 피팅, 통계 테스트 수행, 데이터 탐색, 시각화를 위한 파이썬 라이브러리이다.

### statsmodels이 포함하는 모델
- 선형 모델, 일반 선형 모델, 로버스트 선형모델
- 선형 혼합 효과 모델
- 분산분석 메서드
- 시계열 처리 및 상태 공간 모델
- 일반적률추정법(GMM)

In [41]:
conda install statsmodels

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




## 12.3.1 선형 모델 예측하기

statsmodels에는 아주 기본적인 선형 회귀 모델(최소제곱법)부터 더 복잡한 선형 회귀 모델(반복 재가중 최소제곱법(IRLS))까지 존재한다.

In [42]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [43]:
# 동일한 난수 발생을 위해 시드 값 고정
rng = np.random.default_rng(seed=12345)

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [44]:
X[:5]

y[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [46]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [47]:
model = sm.OLS(y, X)

In [48]:
results = model.fit()
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [51]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Mon, 18 Dec 2023   Prob (F-statistic):                    2.66e-13
Time:                        11:21:24   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [52]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [53]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params
results.tvalues

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [54]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

## 12.3.2 시계열 처리 예측하기

시계열 분석모델에는 자기 회귀 처리, 칼만 필터, 다른 상태 공간 모델, 다변 자기 회귀 모델 등이 있다.

In [55]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [57]:
from statsmodels.tsa.ar_model import AutoReg

MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

In [58]:
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

# 12.4 사이킷런 소개

사이킷런은 가장 널리 사용되는 범용 파이썬 머신러닝 툴킷이다.

사이킷런은 표준적인 지도 학습과 비지도 학습 메서드를 포함하고 있으며  
모델 선택, 평가, 데이터 변형, 데이터 적재, 모델 유지 및 기타 작업을 위한 도구를 제공한다.

In [59]:
conda install scikit-learn


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




In [60]:
train = pd.read_csv('titanic_train.csv')
test = pd.read_csv('titanic_test.csv')
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [61]:
train.isna().sum()
test.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

나이(Age)를 기반으로 생존 여부를 예측하기 위해 누락된 데이터를 학습 데이터셋의 중간값으로 채워 넣는 결측치 처리를 해준다.

In [62]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [63]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [64]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].to_numpy()
X_test = test[predictors].to_numpy()
y_train = train['Survived'].to_numpy()

X_train[:5]
y_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

array([0, 1, 1, 1, 0], dtype=int64)

In [65]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [67]:
model.fit(X_train, y_train)

LogisticRegression()

In [68]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)